In [1]:
# !pip install imblearn

In [36]:
import time, datetime
import requests 
import pickle as pkl 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
import json

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn import metrics
import time
from sklearn.model_selection import ShuffleSplit

import itertools

import warnings
warnings.filterwarnings('ignore')


from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.model_selection import train_test_split

import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from scipy import stats
import pprint
import seaborn as sns

#models 
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.ensemble import VotingClassifier  

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from collections import Counter

import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import StratifiedShuffleSplit

from vecstack import stacking
from xgboost import XGBClassifier


In [27]:
# Prepare data
# data = pd.read_csv("mts_june_10m_rh.csv", dtype='float').drop(['Unnamed: 0'], axis=1)

# X = data.drop(['cs'], axis=1)
# Y = data['cs']

data = pd.read_pickle(r'all_metrics_data_3h.pkl')
# data = pd.read_pickle(r'all_metrics_data_12h.pkl')

X = data.drop(['time', 'label', "txn_fail_num"], axis=1)
Y = data.label

In [28]:
# Preproces for PCA
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(X)

# Create PCA
pca = PCA(n_components = 0.95)
pca_df = pca.fit_transform(scaled_df)
pca.fit(X)

PCA(n_components=0.95)

In [157]:
class FMSAD:
    def __init__(self, X, Y, supervised_models, unsupervised_models):
        self.X = X
        self.Y = Y
        self.supervised_models =supervised_models
        self.unsupervised_models =unsupervised_models
        self.all_models = unsupervised_models+supervised_models
    
    def correlation_analysis(self, plot_heatmap=False):
        '''Calculate label correlation with respect to output label'''
        correlation_txn_falure = {}

        for label in self.X.columns:
            r,p = stats.pearsonr(self.X[label],self.Y)
            correlation_txn_falure[label] = (r,p)
            
            txn_falure_correlation = pd.DataFrame.from_dict(correlation_txn_falure, orient='index').rename(columns={0:'r', 1:'p-value'}).sort_values('r', ascending=False)
        
        if plot_heatmap:
            plt.figure(figsize=(10,6))
            sns.heatmap(txn_falure_correlation, cmap='GnBu', square=True, annot=True, linewidths=.5)
            
        df_index = list(txn_falure_correlation.index)

        txn_falure_correlation = txn_falure_correlation.dropna(axis='rows')
        return txn_falure_correlation
    
    def correlation_features(self, r_lim=0.5, p_lim=0.05):
        '''Select features based specified boundaries'''
        txn_falure_correlation = self.correlation_analysis()
        selected_features = txn_falure_correlation[np.abs(txn_falure_correlation.r) >= r_lim]
        selected_features = selected_features[selected_features['p-value'] <= p_lim]

        selected_features_names = selected_features.index
        return selected_features_names
    
    def evaluate(self, y_real, y_pred):
        from sklearn import metrics
        accuracy = accuracy_score(y_real,y_pred)
        precision = precision_score(y_real, y_pred, average='macro')
        recall = metrics.recall_score(y_real, y_pred, average='macro')
        f1_score = metrics.f1_score(y_real, y_pred, average='macro') 

        # AUC curve
        fpr, tpr, threshold = metrics.roc_curve(y_real, y_pred)
        roc_auc = metrics.auc(fpr, tpr)
        return accuracy, precision, recall, f1_score, fpr, tpr, roc_auc

    # since boolean predictions may be wrong way around
    def result(self, y_real, y_pred):
        from sklearn import metrics
        if  metrics.f1_score(y_real, y_pred, average='macro') >  metrics.f1_score(y_real, [not y for y in y_pred], average='macro'):
            return self.evaluate(y_real, y_pred)
        else:
            return self.evaluate(y_real, [not y for y in y_pred])

    def show_res(self, res):
        accuracy, precision, recall, f1_score, time = res
        print("Accuracy", accuracy)
        print("Precision", precision)
        print("Recall", recall)
        print("F1 Score", f1_score)
        print("Time", time)

    def flip_if_inverted(self, Y_pred):
        Y_pred = np.array([0 if i != 1 else 1 for i in Y_pred ])
        if len(Y_pred[Y_pred == 1]) < len(Y_pred[Y_pred == 0]):

            return [not elem for elem in Y_pred]
        else:
            return Y_pred

    def validate_model(self, model,undersample=False, oversample=False, flip=True):
        accuracy_, precision_, recall_, f1_score_,fpr_, tpr_, roc_auc_, time_ = [],[],[],[],[], [],[],[]

        shuffle_split = StratifiedShuffleSplit(n_splits=5, test_size=0.4, random_state=0)
        for train_index, test_index in shuffle_split.split(self.X,self.Y):
            start_time = time.time()

            if type(X) == pd.DataFrame:
                X_train, X_test = self.X.iloc[train_index], self.X.iloc[test_index]
                Y_train, Y_test = self.Y.iloc[train_index], self.Y.iloc[test_index]
            else:
                X_train, X_test = self.X[train_index], self.X[test_index]
                Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            if undersample:
                undersampling = RandomUnderSampler(sampling_strategy="majority")
                X_train, Y_train = undersampling.fit_resample(X_train, Y_train)

            if oversample:
                oversampling = RandomOverSampler(sampling_strategy="minority")
                X_train, Y_train = oversampling.fit_resample(X_train, Y_train)

            fitted_model = model.fit(X_train, Y_train)
            try:
                y_pred = fitted_model.predict(X_test)
            except:
                y_pred = fitted_model.fit_predict(X_test)

            if flip:
                y_pred = self.flip_if_inverted(y_pred)

            accuracy, precision, recall, f1_score, fpr, tpr, roc_auc = self.result(Y_test, y_pred)

            end_time = time.time()
            fpr_.append(fpr)
            tpr_.append(tpr)

            accuracy_.append(accuracy)
            precision_.append(precision)
            recall_.append(recall)
            f1_score_.append(f1_score)
            roc_auc_.append(roc_auc)
            time_.append(end_time - start_time)              
        return np.mean(accuracy_), np.mean(precision_), np.mean(recall_), np.mean(f1_score_), np.mean(time_)

    def optimize_voting_parameters(self, w1, w2,w3):
        tuning_res = pd.DataFrame(columns = ["w1", "w2", 'w3', "accuracy", "precision", "recall",\
                                             "f1_score", "fpr", "tpr", "roc_auc", "time"])

        for w1_, w2_, w3_ in itertools.product(*[w1, w2,w3]):
            dbscan_clf = DBSCAN(eps=3, min_samples=2)
            if_clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                                    max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
            kmeans_clf = KMeans(n_clusters=2)
            KNN_clf = KNeighborsClassifier(n_neighbors=2)
            svm_clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
            rf_clf = RandomForestClassifier(max_depth=10, random_state=5)      

            # dbscan, IF and kmeans does not work since it is no classifier
            model = VotingClassifier(estimators=[
                    ('knn', KNN_clf), \
                    ('svm', svm_clf),('random forest', rf_clf)], voting='hard', weights=[w1_, w2_, w3_])
            self.validate_model(model)

            accuracy, precision, recall, f1_score, fpr, tpr, roc_auc, time = \
                    self.validate_model(model)

            row = {"w1":w1_, "w2":w2_, 'w3':w3_,"accuracy":accuracy, "precision":precision,\
                   "recall":recall, "f1_score":f1_score, "fpr":fpr, "tpr":tpr, "roc_auc":roc_auc}
            tuning_res = tuning_res.append(row, ignore_index=True)
        return tuning_res.sort_values("f1_score", ascending=False)
    
    
    def weighted_vote_ensamble(self, models, weights=[1,1,1,1,1,1], undersample=False, oversample=False):
        accuracy_, precision_, recall_, f1_score_,fpr_, tpr_, roc_auc_, time_ = [],[],[],[],[], [],[],[]

        shuffle_split = StratifiedShuffleSplit(n_splits=5, test_size=0.4, random_state=0)
        for train_index, test_index in shuffle_split.split(self.X,self.Y):
            start_time = time.time()

            if type(X) == pd.DataFrame:
                X_train, X_test = self.X.iloc[train_index],self.X.iloc[test_index]
                Y_train, Y_test = self.Y.iloc[train_index], self.Y.iloc[test_index]
            else:
                X_train, X_test = self.X[train_index], self.X[test_index]
                Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            if undersample:
                undersampling = RandomUnderSampler(sampling_strategy="majority")
                X_train, Y_train = undersampling.fit_resample(X_train, Y_train)

            if oversample:
                oversampling = RandomOverSampler(sampling_strategy="minority")
                X_train, Y_train = oversampling.fit_resample(X_train, Y_train)


            start_time = time.time()

            results = []
            for model, weight in zip(models, weights):
                try:
                    model.fit(X_train, Y_train)
                    for i in range(weight):
                        res = model.predict(X_test)
                        res = self.flip_if_inverted(res)
                        results.append(res)
                except:
                    for i in range(weight):
                        res =  model.fit_predict(X_test)
                        res[res == -1] = 1
                        res = self.flip_if_inverted(res)
                        results.append(res)

            results = np.array(results)      
            # find majority
            results=results.T

            final_res = []
            for i in list(results):
                final_res.append(int(Counter(i).most_common(1)[0][0]))
            end_time = time.time()

            accuracy, precision, recall, f1_score, fpr, tpr, roc_auc = self.result(Y_test, final_res)

            fpr_.append(fpr)
            tpr_.append(tpr)

            accuracy_.append(accuracy)
            precision_.append(precision)
            recall_.append(recall)
            f1_score_.append(f1_score)
            roc_auc_.append(roc_auc)
            time_.append(end_time - start_time)  

        return np.mean(accuracy_), np.mean(precision_), np.mean(recall_), np.mean(f1_score_), np.mean(time_)
    
    def optimize_all_voting_parameters(self, weight_permutations, undersample=False, oversample=False):
        tuning_res = pd.DataFrame(columns = ["w1", "w2", 'w3', 'w4', 'w5','w6', "accuracy", "precision", "recall",\
                                             "f1_score", "fpr", "tpr", "roc_auc", "time"])

        for permutation in weight_permutations:
            start_time = time.time()
            w1_, w2_, w3_, w4_,w5_,w6_ = permutation

            accuracy, precision, recall, f1_score, fpr, tpr, roc_auc, time_ = \
                self.weighted_vote_ensamble(weights=list(permutation), undersample=undersample, oversample=oversample)


            end_time=time.time()
            row = {"w1":w1_, "w2":w2_, 'w3':w3_,'w4':w4_,'w5':w5_, 'w6':w6_,"accuracy":accuracy, "precision":precision,\
                   "recall":recall, "f1_score":f1_score, "fpr":fpr, "tpr":tpr, "roc_auc":roc_auc, 'time':end_time-start_time}
            tuning_res = tuning_res.append(row, ignore_index=True)
        return tuning_res.sort_values("f1_score", ascending=False)


    def stack(self, models, undersample=False, oversample=False):
        X_train, X_test, Y_train, Y_test = train_test_split(self.X,self.Y, test_size=0.4, random_state=22)

        if undersample:
            undersampling = RandomUnderSampler(sampling_strategy="majority")
            X_train, Y_train = undersampling.fit_resample(X_train, Y_train)

        if oversample:
            oversamping = RandomOverSampler(sampling_strategy="minority")
            X_train, Y_train = oversamping.fit_resample(X_train, Y_train)

        start_time = time.time()
        S_train, S_test = stacking(models,                   
                                    X_train, Y_train, X_test,   
                                   regression=False, 

                                    mode='oof_pred_bag', 

                                   needs_proba=False,

                                   save_dir=None, 

                                   metric=accuracy_score, 

                                   n_folds=4, 

                                   stratified=True,

                                   shuffle=True,  

                                   random_state=0,    

                                   verbose=2)

        model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1,
                              n_estimators=100, max_depth=3)
        model = model.fit(S_train, Y_train)

        y_pred = model.predict(S_test)

        accuracy, precision, recall, f1_score, fpr, tpr, roc_auc = self.result(Y_test, y_pred)

        end_time = time.time()

        return accuracy, precision, recall, f1_score, end_time - start_time

    def generate_models(self, weights, undersample=False, oversample=False):
        df = pd.DataFrame(columns=['model','accuracy', 'precision', 'recall', 'f1 score', 'time'])
        for model in all_models:
            accuracy, precision, recall, f1_score, time = self.validate_model(model, undersample=undersample, \
                                                                            oversample=oversample)
            df = df.append({'model':model,'accuracy': accuracy, 'precision':precision, 'recall':recall, \
                            'f1 score':f1_score, 'time':time}, ignore_index=True)
            
        #ensemble models
#         for i,j in zip([self.unsupervised_models[1:], self.supervised_models, self.all_models[1:]],\
#                             ['unsupervised stacking', 'supervised stacking', 'all model stacking']):
#             accuracy, precision, recall, f1_score, time = self.stack(i, \
#                                                                     undersample=undersample, \
#                                                                     oversample=oversample)
#             df = df.append({'model':j,'accuracy': accuracy, 'precision':precision, 'recall':recall, \
#                                 'f1 score':f1_score, 'time':time}, ignore_index=True)

        for i,j in zip([self.unsupervised_models[1:], self.supervised_models, self.all_models[1:]],\
                            ['unsupervised voting', 'supervised voting', 'all model voting']):
            accuracy, precision, recall, f1_score, time = self.weighted_vote_ensamble(models=i, weights=[1,1,1],\
                                                                                     oversample=oversample,\
                                                                                     undersample=undersample)
            df = df.append({'model':j,'accuracy': accuracy, 'precision':precision, 'recall':recall, \
                                'f1 score':f1_score, 'time':time}, ignore_index=True)
            
        for i,j,z in zip([self.unsupervised_models[1:], self.supervised_models, self.all_models[1:], weights],\
                            ['unsupervised weighted voting', 'supervised weighted voting', 'all model weighted voting'],\
                            weights):
            accuracy, precision, recall, f1_score, time = self.weighted_vote_ensamble(models=i, weights=z,\
                                                                                     oversample=oversample,\
                                                                                     undersample=undersample)
            df = df.append({'model':j,'accuracy': accuracy, 'precision':precision, 'recall':recall, \
                                'f1 score':f1_score, 'time':time}, ignore_index=True)

        return df.sort_values(by="f1 score", ascending=False)

In [158]:
# define all models 

In [159]:
dbscan_model = DBSCAN(eps=3, min_samples=2)
isolation_forest_model = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
kmeans_model = KMeans(n_clusters=2)
knn_model = KNeighborsClassifier()
svm_model = make_pipeline(StandardScaler(), SVC(gamma='auto'))
rf_model = RandomForestClassifier(max_depth=10, random_state=5)

all_models = [dbscan_model, isolation_forest_model, kmeans_model, knn_model, svm_model, rf_model]
unsupervised_models = [dbscan_model, isolation_forest_model, kmeans_model]
supervised_models = [knn_model, svm_model, rf_model]

In [160]:
fmsad = FMSAD(X,Y, supervised_models, unsupervised_models)
fmsad.correlation_analysis()
selected_feature_names = fmsad.correlation_features()

In [161]:
# Create 3 different features selection approaches
df1 = X
df2 = X[selected_feature_names].copy()
df3 = pca_df.copy()

In [162]:
# fmsad.validate_model(isolation_forest_model)
fmsad.generate_models([[0,3,2],[0,3,2],[3,1,1,3,2,2]])

,model,accuracy,precision,recall,f1 score,time
4,"(StandardScaler(), SVC(gamma='auto'))",0.966438,0.964713,0.968236,0.966078,0.031485
10,supervised weighted voting,0.966438,0.964713,0.968236,0.966078,0.199090
5,"(DecisionTreeClassifier(max_depth=10, max_feat...",0.963699,0.962400,0.965111,0.963284,0.152700
7,supervised voting,0.963014,0.961252,0.965358,0.962652,0.246604
11,all model weighted voting,0.956164,0.955284,0.956002,0.955542,0.878958
3,KNeighborsClassifier(),0.943151,0.941690,0.944074,0.942489,0.035912
8,all model voting,0.708219,0.743639,0.678506,0.673475,0.556486
2,KMeans(n_clusters=2),0.552055,0.584593,0.574981,0.545691,0.120523
9,unsupervised weighted voting,0.552055,0.584593,0.574981,0.545691,0.462887
1,"(ExtraTreeRegressor(max_depth=8, max_features=...",0.628767,0.680628,0.584623,0.542958,0.351893
